In [37]:
import requests
import pandas as pd
from io import StringIO 
from bs4 import BeautifulSoup as bs
import re #正規表達法
import time

def countdown(t):
    while t > -1:
        min, sec = divmod(t, 60)
        if t > 0:
            print("{:02d}:{:02d}".format(min, sec), end = '\r')
        elif t == 0:
            print("{:02d}:{:02d}".format(min, sec), end = '\n')
        t -= 1
        time.sleep(1)

url = "https://mops.twse.com.tw/mops/web/ajax_t05st01"
payload = {
            'encodeURIComponent': 1,
            'step': 1,
            'firstin': 1,
            'off': 1,
            'queryName': 'co_id',
            'inpuType': 'co_id',
            'TYPEK': 'all',
            'co_id': 2330,
            'year': 113,

}
while True:
    try:
        res = requests.get(url, params = payload) #發送請求
        res.encoding = 'utf8'
        if res.text.find('Overrun - 查詢過於頻繁,請稍後再試!!') > -1:
            raise Exception('Overrun - 查詢過於頻繁,請稍後再試!!')
        break
    except Exception as e:
        print(e)
        countdown(10)
        continue

res = requests.get(url, params = payload)
df = pd.read_html(StringIO(res.text))[1] #指定標題列

#隱藏版參數
payload = {}
soup = bs(res.text, 'lxml')
inp = soup.select('input[type=hidden]')
for ele in inp:
    if ele.get('value', 999)!= 999:
        payload[ele['name']] = ele['value']
    else:
        payload[ele['name']] = ""

data = []
#透過CSS選擇器，抓取表格按鈕內的隱藏參數
btns = soup.select(".hasBorder input")
for btn in btns:
    m = re.findall(r"t05st01_fm\.(\w+)\.value='(\w+)'", btn["onclick"])
    #合併表單隱藏參數與按鈕參數
    print(dict(m))
    payload.update(dict(m))   #串列轉字典，與上面payload合併
    #重複執行

    while True:
        try:
            res = requests.get(url, params = payload) #發送請求
            res.encoding = 'utf8'
            if res.text.find('Overrun - 查詢過於頻繁,請稍後再試!!') > -1:
                raise Exception('Overrun - 查詢過於頻繁,請稍後再試!!')
            break
        except Exception as e:
            print(e)
            countdown(10)
            continue
        
    
    soup = bs(res.text, 'lxml')
    #抓取”說明“
    data.append(soup.select('pre')[1].text)
del df['Unnamed: 5']
df.insert(5, "說明", data)
df

{'seq_no': '1', 'spoke_time': '160807', 'spoke_date': '20240103', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '163628', 'spoke_date': '20240104', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '171530', 'spoke_date': '20240105', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '141001', 'spoke_date': '20240110', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '171537', 'spoke_date': '20240117', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '154253', 'spoke_date': '20240118', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '2', 'spoke_time': '165123', 'spoke_date': '20240118', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '173502', 'spoke_date': '20240119', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '175742', 'spoke_date': '20240124', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no': '1', 'spoke_time': '162551', 'spoke_date': '20240131', 'co_id': '2330', 'TYPEK': 'sii'}
{'seq_no':

,公司代號,公司名稱,發言日期,發言時間,主旨,說明
0,2330,台積電,113/01/03,16:08:07,公告本公司112年第四季法人說明會將於113年1月18日召開,\r\n符合條款第四條第XX款：12\r\n事實發生日：113/01/18\r\n1.召開法...
1,2330,台積電,113/01/04,16:36:28,本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,\r\n1.證券名稱:\r\n公司債。\r\n2.交易日期:112/10/3~113/1/4...
2,2330,台積電,113/01/05,17:15:30,本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,\r\n1.證券名稱:\r\n公司債。\r\n2.交易日期:112/12/18~113/1/...
3,2330,台積電,113/01/10,14:10:01,台積公司2023年12月營收報告,\r\n1.事實發生日:113/01/10\n2.公司名稱:台灣積體電路製造股份有限公司\n...
4,2330,台積電,113/01/17,17:15:37,本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,\r\n1.證券名稱:\r\n政府機構不動產抵押貸款債券。\r\n2.交易日期:113/1/...
...,...,...,...,...,...,...
218,2330,台積電,113/10/23,17:53:45,本公司代子公司 TSMC Global Ltd. 公告處分固定收益證券,"\r\n1.證券名稱:\r\n公司債, 政府機構不動產抵押貸款債券。\r\n2.交易日期:1..."
219,2330,台積電,113/10/24,17:23:01,本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,\r\n1.證券名稱:\r\n公司債。\r\n2.交易日期:113/10/9~113/10/...
220,2330,台積電,113/10/24,17:27:32,本公司代子公司 TSMC Global Ltd. 公告處分固定收益證券,\r\n1.證券名稱:\r\n政府機構不動產抵押貸款債券。\r\n2.交易日期:113/10...
221,2330,台積電,113/10/25,17:03:17,"本公司代子公司Growth Fund Limited公告處分 Astera Labs, In...","\r\n1.證券名稱:\r\nAstera Labs, Inc. (""Astera"")\r\..."


In [16]:
import requests
import pandas as pd
from io import StringIO 
from bs4 import BeautifulSoup as bs

url = "https://mops.twse.com.tw/mops/web/ajax_t05st01"
payload = {
            'encodeURIComponent': 1,
            'step': 1,
            'firstin': 1,
            'off': 1,
            'queryName': 'co_id',
            'inpuType': 'co_id',
            'TYPEK': 'all',
            'co_id': 2330,
            'year': 113,

}
res = requests.get(url, params = payload)
df = pd.read_html(StringIO(res.text)) [1]#指定標題列
btns = soup.select('.hasBorder input')
for btn in btns:
    print(btn['onclick'])

document.t05st01_fm.action='ajax_t05st01';document.t05st01_fm.seq_no.value='1';document.t05st01_fm.spoke_time.value='160807';document.t05st01_fm.spoke_date.value='20240103';document.t05st01_fm.co_id.value='2330';document.t05st01_fm.TYPEK.value='sii';openWindow(this.form ,'');
document.t05st01_fm.action='ajax_t05st01';document.t05st01_fm.seq_no.value='1';document.t05st01_fm.spoke_time.value='163628';document.t05st01_fm.spoke_date.value='20240104';document.t05st01_fm.co_id.value='2330';document.t05st01_fm.TYPEK.value='sii';openWindow(this.form ,'');
document.t05st01_fm.action='ajax_t05st01';document.t05st01_fm.seq_no.value='1';document.t05st01_fm.spoke_time.value='171530';document.t05st01_fm.spoke_date.value='20240105';document.t05st01_fm.co_id.value='2330';document.t05st01_fm.TYPEK.value='sii';openWindow(this.form ,'');
document.t05st01_fm.action='ajax_t05st01';document.t05st01_fm.seq_no.value='1';document.t05st01_fm.spoke_time.value='141001';document.t05st01_fm.spoke_date.value='202401

In [24]:
import re
btns = soup.select(".hasBorder input")
m = re.findall(r"t05st01_fm\.(\w+)\.value='(\w+)'",btns[0]["onclick"])
dict(m) #串列轉字典

{'seq_no': '1',
 'spoke_time': '160807',
 'spoke_date': '20240103',
 'co_id': '2330',
 'TYPEK': 'sii'}